In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold

from tqdm import tqdm
from random import choices


import kerastuner as kt

import gc
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold

In [ ]:
##前處理參考Bottleneck encoder + MLP + Keras Tuner 8601c5
#Train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
##Pandas query
##https://blog.csdn.net/AlanGuoo/article/details/88874742
##reset_index()
##https://ithelp.ithome.com.tw/articles/10194006
#Train = Train.query('date > 85').reset_index(drop = True)
##篩選出date>85
##reset_index()可以讓index重置成原本的樣子
##轉型態astype
##https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html
#Train = Train.astype({c: np.float32 for c in Train.select_dtypes(include='float64').columns}) #limit memory use
##dtype選擇特定條件之列:資料型別float64
##float64->float32
##fillna填0，mean()會自動忽略nan，inplace=True會取代原數據，就不用再丟回，ex:Train()，Train = Train.fillna(Train.mean())
#Train.fillna(Train.mean(),inplace=True)
#Train = Train.query('weight > 0').reset_index(drop = True)
#Train['action'] =  (  (Train['resp_1'] > 0 ) & (Train['resp_2'] > 0 ) & (Train['resp_3'] > 0 ) & (Train['resp_4'] > 0 ) &  (Train['resp'] > 0 )   ).astype('int')
##原數據有小數float->int

#features = [c for c in Train.columns if 'feature' in c]#抓出特徵

#resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']#抓出resp

#X = Train[features].values #用來訓練
#y = np.stack([(Train[c] > 0).astype('int') for c in resp_cols]).T #Multitarget

#f_mean = np.mean(Train[features[1:]].values,axis=0)#mean()會自動忽略nan，抓出有值的部分，索引1到最後
##不抓features_0

In [ ]:
#features_CSV = pd.read_csv('../input/jane-street-market-prediction/features.csv')#
##https://www.ycc.idv.tw/tensorflow-tutorial_4.html
##features.csv抓features_0查看數值意義
##f_features = np.mean(features_CSV[0:1].values,axis=0)

#f_features = features_CSV[0:1].values
#f_features = np.delete(f_features,[0][0])
#print(f_features)
#f_features = f_features.astype('int')
#f_features

##在features.csv中features_0數值都一樣，無意義，因此在train.csv裡沒用到

In [ ]:
#減少記憶體空間
#Jane Street - Tensorflow Dense

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train = reduce_mem_usage(train)
features = [c for c in train.columns if 'feature' in c]#抓出特徵

NAN_VALUE = -999#空值填入-999

In [ ]:
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float16').columns}) 
train = train.fillna(train.mean())
##dtype選擇特定條件之列:資料型別float16
##float16->float32
##fillna填0，mean()會自動忽略nan，inplace=True會取代原數據，就不用再丟回，ex:Train()，Train = Train.fillna(Train.mean())
f_mean = np.mean(train[features[1:]].values,axis=0)
#mean()會自動忽略nan，抓出有值的部分，索引1到最後
##不抓features_0
train = train.query('date > 85').reset_index(drop = True)
##篩選出date>85
##reset_index()可以讓index重置成原本的樣子
train = train[train.weight != 0]
#0無意義
n_folds = 5
#折數
seed = 2020
#種子數
skf = StratifiedKFold(n_splits=n_folds, shuffle=False)
#shuffle隨機排序

X = train.loc[:, features].values
#if np.isnan(X[:, 1:].sum()):
#    X[:, 1:] = np.nan_to_num(X[:, 1:]) + np.isnan(X[:, 1:]) * NAN_VALUE
    
#y = (train['resp'].values > 0).astype(int)
resp_cols = ['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']#抓出resp
y = np.stack([(train[c] > 0.000001).astype('int') for c in resp_cols]).T 

train_index, test_index = next(skf.split(X, y[:,0]))
#train_index, test_index = next(skf.split(X, y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

In [ ]:
#Jane Street - Tensorflow Dense
TUNNING = False
#以下為tf.keras.layers中之應用的解釋
#keras的layer類直接建立深度網絡中的layer
##Input用於實例化Keras
##BatchNormalization在每批中對上一層的激活進行歸一化，即應用一個轉換，將平均激活保持在0附近並將激活標準偏差保持在1附近
##GaussianNoise高斯噪聲
##Dropout輸入
###http://man.hubwiz.com/docset/TensorFlow.docset/Contents/Resources/Documents/api_docs/python/tf/keras/layers/BatchNormalization.html
##Dense用於添加一個全連接層
def create_model(hp,input_dim,output_dim):
    inputs = tf.keras.layers.Input(input_dim)
    x = tf.keras.layers.BatchNormalization()(inputs)
    x = tf.keras.layers.GaussianNoise(hp.Choice('noise',[0.0,0.03,0.05]))(x)
    x = tf.keras.layers.Dropout(hp.Choice('init_dropout',[0.0,0.3,0.5]))(x)    
    x = tf.keras.layers.Dense(hp.Int('num_units_1', 128, 2048, 64), activation=hp.Choice('activation_1', ['tanh','relu','swish']))(x)
    #units輸出變數
    x = tf.keras.layers.Dropout(hp.Choice(f'dropout_1',[0.0,0.3,0.5]))(x)
    x = tf.keras.layers.Dense(hp.Int('num_units_2', 128, 1024, 32), activation=hp.Choice('activation_2', ['tanh','relu','swish']))(x)
    x = tf.keras.layers.Dropout(hp.Choice(f'dropout_2',[0.0,0.3,0.5]))(x)
    x = tf.keras.layers.Dense(output_dim, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=inputs,outputs=x)#設定輸出入之格式
    #compile編譯模型
    #解釋：https://dotblogs.com.tw/greengem/2017/12/17/094023
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('lr',[1e-2, 1e-3, 1e-5])),
                  #optimizer優化器選用
                  #實現Adam算法的優化器
                  #優化器解釋：https://keras.io/zh/optimizers/
                  #lr為學習率
                  loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=hp.Choice('label_smoothing',[0.0, 0.01, 0.1])),
                  #tf.keras.losses.BinaryCrossentropy計算真實標籤和預測標籤之間的交叉熵損失
                  metrics=[tf.keras.metrics.AUC(name = 'auc')])
                  #metrics監視模型並判斷性能
                  #tf.keras.metrics.AUC通過黎曼和求出近似的AUC（曲線下的面積）
    return model
#Sequential將線性的層堆疊到一個tf.keras.Model
#https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
#創建一個“Sequential”模型並添加一個Dense層作為第一層
model = tf.keras.Sequential([
    tf.keras.Input(shape = len(features)),#特徵長度
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GaussianNoise(0.05),
    tf.keras.layers.Dropout(0.00), 
    tf.keras.layers.Dense(2048, activation='tanh'),#雙曲正切激活函數
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1024, activation='swish'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2048, activation='swish'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='swish'),    
    tf.keras.layers.Dense(5, activation = 'sigmoid')#Sigmoid 激活函数
  ])
#activation的相關解釋：https://keras.io/zh/activations/
#tf.keras.layers.Dense(輸出入尺寸)
EPOCHS = 500#訓練過程中數據將被用多少次
BATCH_SIZE = 1024#batch_size=4096,#梯度下降

if TUNNING:
    import kerastuner as kt
    EPOCHS = 50#訓練過程中數據將被用多少次
    MAX_TRIAL = 20#調參過程中進行實驗的參數組合的總數目
    model_fn = lambda hp: create_model(hp, X_train.shape[-1], y_train.shape[-1])
    tuner = kt.tuners.BayesianOptimization(model_fn, kt.Objective('val_auc', direction='max'), MAX_TRIAL, seed = 2020)
    #貝葉斯優化
    tuner.search(X_train, y_train, epochs=EPOCHS, validation_data=(X_test, y_test),callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10, restore_best_weights=True)])
    model = tuner.get_best_models()[0]
else:
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    #optimizer = tf.keras.optimizers.RMSprop()
    loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.00)
    model.compile(loss = loss, optimizer=optimizer, metrics=[tf.keras.metrics.AUC()])
    #觀察網絡性能指標
    history = model.fit(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[callback], validation_data=(X_test, y_test))

In [ ]:
if TUNNING:
    #打印結果摘要
    tuner.results_summary()

In [ ]:
import janestreet
from tqdm.notebook import tqdm
#janestreet.competition.make_env.__called__ = False
env = janestreet.make_env()
#啟動環境，JaneStreet使用該環境提供測試行。然後，它創建一個迭代器，該迭代器允許獲取您需要預測的所有行。
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in tqdm(iter_test):
    #循環拋出所有應該預測的行。每個test_df是一個數據框，其中包含必須預測的一行。
    if test_df['weight'].item() > 0:
        #如果權重為0，則預測將對分數沒有影響。因此，為了節省計算時間，我們不預測權重== 0的時間
        x_tt = test_df.loc[:, features].values
        #如前所述，每個test_df都包含一個數據幀。但是您的模型無法使用數據框，它需要一個數組數組……然後將其轉換
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
            #x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * NAN_VALUE
            #如果缺少任何值，我們必須對其進行編輯。在這種情況下，我不確定目標是什麼。但是，填充缺失值的一種好
            #方法是用平均值（對異常值敏感），中位數（對異常值不敏感）或默認值（0）代替它們。
        action = np.mean(model(x_tt, training = False).numpy()[0])#模型結果
        if (action > 0.5):
            sample_prediction_df.action = 1
        else:
            sample_prediction_df.action = 0 
        #等同
        #pred = f(pred)
        #sample_prediction_df.action = np.where(pred >= th, 1, 0).astype(int)#只是沒有=
        ##預測動作是pred內容（將其視為具有類別0或1的概率）
    else:
        sample_prediction_df.action = 0 
    env.predict(sample_prediction_df)

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
class CVTuner(kt.engine.tuner.Tuner):
    def run_trial(self, trial, X, y, splits, batch_size=32, epochs=1,callbacks=None):
        val_losses = []
        for train_indices, test_indices in splits:
            X_train, X_test = [x[train_indices] for x in X], [x[test_indices] for x in X]
            y_train, y_test = [a[train_indices] for a in y], [a[test_indices] for a in y]
            if len(X_train) < 2:
                X_train = X_train[0]
                X_test = X_test[0]
            if len(y_train) < 2:
                y_train = y_train[0]
                y_test = y_test[0]
            
            model = self.hypermodel.build(trial.hyperparameters)
            hist = model.fit(X_train,y_train,
                      validation_data=(X_test,y_test),
                      epochs=epochs,
                        batch_size=batch_size,
                      callbacks=callbacks)
            
            val_losses.append([hist.history[k][-1] for k in hist.history])
        val_losses = np.asarray(val_losses)
        self.oracle.update_trial(trial.trial_id, {k:np.mean(val_losses[:,i]) for i,k in enumerate(hist.history.keys())})
        self.save_model(trial.trial_id, model)

In [ ]:
#model_fn = lambda hp: create_model(hp,X.shape[-1],y.shape[-1],encoder)

#tuner = CVTuner(
#        hypermodel=model_fn,
#        oracle=kt.oracles.BayesianOptimization(
#        objective= kt.Objective('val_auc', direction='max'),
#        num_initial_points=4,
#        max_trials=20))

#FOLDS = 5
#SEED = 42

#if TRAINING:
#    gkf = PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap=20)
#    splits = list(gkf.split(y, groups=train['date'].values))
#    tuner.search((X,),(y,),splits=splits,batch_size=4096,epochs=100,callbacks=[EarlyStopping('val_auc', mode='max',patience=3)])
#    #hp  = tuner.get_best_hyperparameters(1)[0]
#    pd.to_pickle(hp,f'./best_hp_{SEED}.pkl')
#    for fold, (train_indices, test_indices) in enumerate(splits):
#        model = model_fn(hp)
#        X_train, X_test = X[train_indices], X[test_indices]
#        y_train, y_test = y[train_indices], y[test_indices]
#        model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=4096,callbacks=[EarlyStopping('val_auc',mode='max',patience=10,restore_best_weights=True)])
#        model.save_weights(f'./model_{SEED}_{fold}.hdf5')
#        model.compile(Adam(hp.get('lr')/100),loss='binary_crossentropy')
#        model.fit(X_test,y_test,epochs=3,batch_size=4096)
#        model.save_weights(f'./model_{SEED}_{fold}_finetune.hdf5')
#    tuner.results_summary()
#else:
    #models = []
    #hp = pd.read_pickle(f'../input/vae223/best_hp_{SEED}.pkl')
    #for f in range(FOLDS):
    #    model = model_fn(hp)
    #    if USE_FINETUNE:
    #        model.load_weights(f'../input/vae223/model_{SEED}_{f}_finetune.hdf5')
    #        model.load_weights(f'./model_{SEED}_{f}_finetune.hdf5')
    #    else:
    #        #model.load_weights(f'../input/vae223/model_{SEED}_{f}.hdf5')
    #        model.load_weights(f'./model_{SEED}_{f}.hdf5')
    #    models.append(model)

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=333) 

#print(X_train.shape, X_test.shape, y_train.shape, y_test.shape) 

In [ ]:
#https://towardsdatascience.com/multiclass-classification-and-information-bottleneck-an-example-using-keras-5591b9a2c000
#Bottleneck
#from keras import models
#from keras import layers

#model = models.Sequential()
#model.add(layers.Dense(64, activation='relu', input_shape = (130,)))#數值為X.shape
#model.add(layers.Dense(64, activation='relu'))#使用具有64個隱藏單元的圖層
#model.add(layers.Dense(5, activation='softmax'))#數值為y.shape

In [ ]:
#編譯模型
#model.compile(optimizer='rmsprop',
#             loss='categorical_crossentropy',
#             metrics=['accuracy'])
#解釋：https://dotblogs.com.tw/greengem/2017/12/17/094023
#優化器選用rmsprop
#損失函數當預測值與實際值愈相近，損失函數就愈小，反之差距很大，就會更影響損失函數的值，這篇文章 主張要用 Cross Entropy 取代 MSE，
#因為，在梯度下時，Cross Entropy 計算速度較快
#監視accuracy模型
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.losses import BinaryCrossentropy
#model.compile(optimizer=Adam(0.001),
#              loss='binary_crossentropy',
#              metrics=[tf.keras.metrics.AUC(name = 'auc')])

In [ ]:
#取得驗證集
# Validation samples
#X_val = X_train[:1000]
#partial_X_train = X_train[1000:]

# Validation labels
#y_val = y_train[:100]
#partial_y_train = y_train[1000:]

In [ ]:
#from keras.callbacks import ModelCheckpoint, EarlyStopping
#訓練模型
#epochs, batch_size, validation_split參考Bottleneck encoder + MLP + Keras Tuner 8601c5
#model_train = model.fit(partial_X_train,
#                   partial_y_train,
#                   epochs=100,#epochs=20#epochs=9
#                   batch_size=4096,#batch_size=512
#                   #epochs=30,#epochs=20#epochs=9
#                   #batch_size=128,#batch_size=512
#                   #validation_split=0.1,
#                   #callbacks=[EarlyStopping('val_loss',patience=10,restore_best_weights=True)]
#                   validation_data=(X_val, y_val)
#                   )
#model_train = model.fit(X_train,y_train,
#              validation_data=(X_test,y_test),#驗證集
#              epochs=1000,#訓練過程中數據將被用多少次
#              batch_size=4096,#梯度下降
#              callbacks=[EarlyStopping('val_auc',mode='max',patience=10,restore_best_weights=True)])
              #回呼函數
    #EarlyStopping是Callbacks的一種，callbacks用於指定在每個epoch開始和結束的時候進行哪種特定操作。
    #Callbacks中有一些設置好的接口，可以直接使用，如’acc’,’val_acc’,’loss’和’val_loss’等等。
    #patience：能夠容忍多少個epoch內都沒有improvement。這個設置其實是在抖動和真正的準確率下降之間做tradeoff。
    #mode: 就’auto’, ‘min’, ‘,max’三個可能。如果知道是要上升還是下降，建議設置一下。monitor是’acc’，所以mode=’max’。
    #restore_best_weights：. 如果要在停止後儲存最佳權重，請將此引數設定為True. 
    #https://www.mdeditor.tw/pl/2oB4/zh-tw
    #https://medium.com/ai%E5%8F%8D%E6%96%97%E5%9F%8E/learning-model-earlystopping%E4%BB%8B%E7%B4%B9-%E8%BD%89%E9%8C%84-f364f4f220fb

In [ ]:
#將模型儲存至 HDF5 檔案中
#https://blog.gtwang.org/programming/keras-save-and-load-model-tutorial/
#model.save(f'./my_model.h5')

In [ ]:
#from keras.models import load_model

# 刪除既有模型變數
#del model 

# 載入模型
#model = load_model('my_model.h5')

# 驗證模型
#score = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
#觀察網絡性能指標
#history_dict = model_train.history
#history_dict.keys()

In [ ]:
#訓練和驗證準確性
#import matplotlib.pyplot as plt
#%matplotlib inline
#loss_values = history_dict['loss']
#val_loss_values = history_dict['val_loss']

#epochs = range(1, len(loss_values) + 1)

#plt.plot(epochs, loss_values, 'bo', label="Training Loss")
#plt.plot(epochs, val_loss_values, 'b', label="Validation Loss")

#plt.title('Training and Validation Loss')
#plt.xlabel('Epochs')
#plt.ylabel('Loss Value')
#plt.legend()

#plt.show()

In [ ]:
# Training and Validation Accuracy

#acc_values = history_dict['accuracy']
#val_acc_values = history_dict['val_accuracy']

#acc_values = history_dict['auc']
#val_acc_values = history_dict['val_auc']

#epochs = range(1, len(loss_values) + 1)

#plt.plot(epochs, acc_values, 'ro', label="Training Accuracy")
#plt.plot(epochs, val_acc_values, 'r', label="Validation Accuracy")

#plt.title('Training and Validation Accuraccy')
#plt.xlabel('Epochs')
#plt.ylabel('Accuracy')
#plt.legend()

#plt.show()

In [ ]:
#results = model.evaluate(X_test, y_test)

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=333)

#print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
#資料分割參考：Jane Street - Tensorflow Dense
#from sklearn.model_selection import StratifiedKFold

#skf = StratifiedKFold(n_splits=5, shuffle=False)

#train_index, test_index = next(skf.split(X, y[:,0]))
#train_index, test_index = next(skf.split(X, y))
#X_train, X_test = X[train_index], X[test_index]
#y_train, y_test = y[train_index], y[test_index]
#print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
#https://github.com/jorgesleonel/Multilayer-Perceptron/blob/master/Basic%20Multi-Layer%20Perceptron.ipynb

#import sklearn
#from sklearn import preprocessing
##from sklearn.model_selection import train_test_split 
#from sklearn.preprocessing import StandardScaler  
#from sklearn.neural_network import MLPClassifier 
#from sklearn.metrics import classification_report, confusion_matrix



In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
#特徵縮放
#scaler = StandardScaler()  
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)  
#X_test = scaler.transform(X_test)

In [ ]:
#MLP多層感測器
#mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)  
#mlp.fit(X_train, y_train.values.ravel())
#mlp.fit(X_train, y_train)

In [ ]:
#預測結果
#predictions = mlp.predict(X_test)
#print(predictions)
#查看效能
#print(confusion_matrix(y_test,predictions))  
#print(confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1)))
#print(classification_report(y_test,predictions))

In [ ]:
#多類分類器與信息瓶頸之分數為0
#改用xgboost->但跑太久或不動
#https://www.itread01.com/articles/1476146171.html
#import xgboost as xgb
#from xgboost.sklearn import XGBClassifier
#https://medium.com/jameslearningnote/%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90-%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-%E7%AC%AC5-2%E8%AC%9B-kaggle%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92%E7%AB%B6%E8%B3%BD%E7%A5%9E%E5%99%A8xgboost%E4%BB%8B%E7%B4%B9-1c8f55cffcc
#xgbc = XGBClassifier()
#xgbc.fit(X_train, y_train)

#pred_xgbc = xgbc.pred(X_test, y_test)

In [ ]:
#https://ithelp.ithome.com.tw/articles/10197461
#交叉驗證
#from sklearn.model_selection import cross_val_score#評分資料準確度的
#from sklearn import datasets
#from sklearn.model_selection import train_test_split
#from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#knn = KNeighborsClassifier(n_neighbors=10)#找尋附近10鄰居
#scores = cross_val_score(knn,X,y,cv=5,scoring='accuracy')#分5組
#print(scores)
#print(scores.mean())

In [ ]:
##https://kegui.medium.com/how-to-do-cross-validation-in-keras-tuner-db4b2dbe079a
#import kerastuner
#from sklearn import model_selection

#class CVTuner(kerastuner.engine.tuner.Tuner):
#    def run_trial(self, trial, x, y, batch_size=32, epochs=1):
#        cv = model_selection.KFold(5)
#        val_losses = []
#        for train_indices, test_indices in cv.split(x):
#            x_train, x_test = x[train_indices], x[test_indices]
#            y_train, y_test = y[train_indices], y[test_indices]
#            model = self.hypermodel.build(trial.hyperparameters)
#            model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
#            val_losses.append(model.evaluate(x_test, y_test))
#        self.oracle.update_trial(trial.trial_id, {'val_loss': np.mean(val_losses)})
#        self.save_model(trial.trial_id, model)

In [ ]:
#def create_model(hp,input_dim,output_dim,encoder):
#    inputs = Input(input_dim)
    
#    x = encoder(inputs)
#    x = Concatenate()([x,inputs]) #use both raw and encoded features
#    x = BatchNormalization()(x)
#    x = Dropout(hp.Float('init_dropout',0.0,0.5))(x)
    
#    for i in range(hp.Int('num_layers',1,3)):
#        x = Dense(hp.Int('num_units_{i}',64,256))(x)
#        x = BatchNormalization()(x)
#        x = Lambda(tf.keras.activations.swish)(x)
#        x = Dropout(hp.Float(f'dropout_{i}',0.0,0.5))(x)
#    x = Dense(output_dim,activation='sigmoid')(x)
#    model = Model(inputs=inputs,outputs=x)
#    model.compile(optimizer=Adam(hp.Float('lr',0.00001,0.1,default=0.001)),loss=BinaryCrossentropy(label_smoothing=hp.Float('label_smoothing',0.0,0.1)),metrics=[tf.keras.metrics.AUC(name = 'auc')])
#    return model

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
#from sklearn.model_selection import KFold
#FOLDS = 5
#kf = KFold(n_splits=FOLDS)
#kf.get_n_splits(X)

#print(kf)

#from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
#from tensorflow.keras.models import Model, Sequential
#from tensorflow.keras.losses import BinaryCrossentropy
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow.keras.layers.experimental.preprocessing import Normalization
#from sklearn.model_selection import GroupKFold

#from tqdm import tqdm
#from random import choices
#import kerastuner as kt
#inputs = Input(X.shape[-1])
##https://keras.io/api/models/model/
#x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)

#model = Model(inputs=inputs,outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x))
#model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
#              loss='BinaryCrossentropy',metrics=[tf.keras.metrics.AUC(name = 'auc')])

#my_model = lambda hp: create_model(hp,X.shape[-1],y.shape[-1],encoder)

#models = []
#fold = 0
#for train_index, test_index in kf.split(X):
#    print("TRAIN:", train_index, "TEST:", test_index)
#    X_train, X_test = X[train_index], X[test_index]
#    y_train, y_test = y[train_index], y[test_index]
    
#    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=4096,callbacks=[EarlyStopping('val_auc',mode='max',patience=10,restore_best_weights=True)])
#    model.save_weights(f'./model_{42}_{fold}.hdf5')
#    model.compile(Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),loss='binary_crossentropy')
#    model.fit(X_test,y_test,epochs=3,batch_size=4096)
#    #model.save_weights(f'./model_{42}_{fold}_finetune.hdf5')
#    models.append(model)
#    fold = fold+1
#for f in range(FOLDS):
#    model = model.load_weights(f'./model_{42}_{f}_finetune.hdf5')
#    models.append(model)

In [ ]:
#models = models[-2:]
#f = np.median
#th = 0.5
#import janestreet
#janestreet.make_env.__called__ = False
#env = janestreet.make_env()
##啟動環境，JaneStreet使用該環境提供測試行。然後，它創建一個迭代器，該迭代器允許獲取您需要預測的所有行。
#for (test_df, sample_prediction_df) in tqdm(env.iter_test()):
##循環拋出所有應該預測的行。每個test_df是一個數據框，其中包含必須預測的一行。
#    if test_df['weight'].item() > 0:
#    #如果權重為0，則預測將對分數沒有影響。因此，為了節省計算時間，我們不預測權重== 0的時間
    
#        X_tt = test_df.loc[:, features].values
#        #如前所述，每個test_df都包含一個數據幀。但是您的模型無法使用數據框，它需要一個數組數組……然後將其轉換
#        if np.isnan(X_tt[:, 1:].sum()):
#            X_tt[:, 1:] = np.nan_to_num(X_tt[:, 1:]) + np.isnan(X_tt[:, 1:]) * f_mean
#            #如果缺少任何值，我們必須對其進行編輯。在這種情況下，我不確定目標是什麼。但是，填充缺失值的一種好
#            #方法是用平均值（對異常值敏感），中位數（對異常值不敏感）或默認值（0）代替它們。
#        pred = np.mean([model(X_tt, training = False).numpy() for model in models],axis=0)
#        #pred = np.mean([model(X_tt, training = False).numpy()],axis=0)#模型
#        #pred = np.mean(predictions,axis=0)#模型
#        pred = f(pred)
#        sample_prediction_df.action = np.where(pred >= th, 1, 0).astype(int)
#        #預測動作是pred內容（將其視為具有類別0或1的概率）
#    else:
#        sample_prediction_df.action = 0
#    env.predict(sample_prediction_df)